# Hand recognition

In [2]:
import cv2
import numpy as np
import time

In [3]:
# Function to find angle between two vectors
def nothing(x):
    pass

def Angle(v1,v2):
    dot = np.dot(v1,v2)
    x_modulus = np.sqrt((v1*v1).sum())
    y_modulus = np.sqrt((v2*v2).sum())
    cos_angle = dot / x_modulus / y_modulus
    angle = np.degrees(np.arccos(cos_angle))
    return angle

# Function to find distance between two points in a list of lists
def FindDistance(A,B): 
    return np.sqrt(np.power((A[0][0]-B[0][0]),2) + np.power((A[0][1]-B[0][1]),2)) 

#Perform morphological transformations to filter out the background noise
def clean_background(frame):
    #Convert to HSV color space
    hsv_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    
    #Create a binary image with where white will be skin colors and rest is black
    mask = cv2.inRange(hsv_frame,np.array([2,50,50]),np.array([15,255,255]))
    
    #Kernel matrices for morphological transformation    
    kernel_square = np.ones((11,11),np.uint8)
    kernel_ellipse= cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    
    #Dilation increase skin color area
    #Erosion increase skin color area
    dilation = cv2.dilate(mask,kernel_ellipse,iterations = 1)
    erosion = cv2.erode(dilation,kernel_square,iterations = 1)    
    dilation2 = cv2.dilate(erosion,kernel_ellipse,iterations = 1)    
    filtered = cv2.medianBlur(dilation2,5)
    kernel_ellipse= cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(8,8))
    dilation2 = cv2.dilate(filtered,kernel_ellipse,iterations = 1)
    kernel_ellipse= cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    dilation3 = cv2.dilate(filtered,kernel_ellipse,iterations = 1)
    median = cv2.medianBlur(dilation2,5)
    ret = cv2.threshold(median,127,255,0)[1]
    cv2.imshow('BGclean',ret)
    return ret

In [ ]:
#Open Camera object
cap = cv2.VideoCapture(0)

# Decrease frame size
# Note this may not work, only depends on camera support
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 200)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 200)
# cap.set(cv2.CAP_PROP_FPS, 12)

# Creating a window for HSV track bars
cv2.namedWindow('TrackBars')
cv2.createTrackbar('Hlower','TrackBars',0,255,nothing)
cv2.createTrackbar('Slower','TrackBars',0,255,nothing)
cv2.createTrackbar('Vlower','TrackBars',0,255,nothing)
cv2.createTrackbar('Hupper','TrackBars',0,255,nothing)
cv2.createTrackbar('Supper','TrackBars',0,255,nothing)
cv2.createTrackbar('Vupper','TrackBars',0,255,nothing)

# Initial values to avoid error:
cv2.setTrackbarPos('Hlower', 'TrackBars', 5)
cv2.setTrackbarPos('Slower', 'TrackBars', 50)
cv2.setTrackbarPos('Vlower', 'TrackBars', 50)
cv2.setTrackbarPos('Hupper', 'TrackBars', 15)
cv2.setTrackbarPos('Supper', 'TrackBars', 255)
cv2.setTrackbarPos('Vupper', 'TrackBars', 255)

while(1):
    start_time = time.time()
    
    Hlower = cv2.getTrackbarPos('Hlower','TrackBars')
    Slower = cv2.getTrackbarPos('Slower','TrackBars')
    Vlower = cv2.getTrackbarPos('Vlower','TrackBars')
    Hupper = cv2.getTrackbarPos('Hupper','TrackBars')
    Supper = cv2.getTrackbarPos('Supper','TrackBars')
    Vupper = cv2.getTrackbarPos('Vupper','TrackBars')
    
    #Capture frame
    frame = cap.read()[1]

    # Resize the image
#     frame = cv2.resize(frame, (0,0), fx=0.5, fy=0.5) 

#     threshold = clean_background(frame) ##################################### Calling original function, return when done
    # create trackbars for color change

    #Convert to HSV color space
    hsv_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    
    #Create a binary image with where white will be skin colors and rest is black
#     mask = cv2.inRange(hsv_frame,np.array([2,50,50]),np.array([15,255,255]))
    mask = cv2.inRange(hsv_frame,np.array([Hlower,Slower,Vlower]),np.array([Hupper,Supper,Vupper]))
    #Kernel matrices for morphological transformation    
    kernel_square = np.ones((11,11),np.uint8)
    kernel_ellipse= cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    
    #Dilation increase skin color area
    #Erosion increase skin color area
    dilation = cv2.dilate(mask,kernel_ellipse,iterations = 1)
    erosion = cv2.erode(dilation,kernel_square,iterations = 1)    
    dilation2 = cv2.dilate(erosion,kernel_ellipse,iterations = 1)    
    filtered = cv2.medianBlur(dilation2,5)
    kernel_ellipse= cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(8,8))
    dilation2 = cv2.dilate(filtered,kernel_ellipse,iterations = 1)
    kernel_ellipse= cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    dilation3 = cv2.dilate(filtered,kernel_ellipse,iterations = 1)
    median = cv2.medianBlur(dilation2,5)
    threshold = cv2.threshold(median,127,255,0)[1]
    cv2.imshow('TrackBars',dilation)
    
    #Find contours of the filtered frame
    _, contours, hierarchy = cv2.findContours(threshold,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)   
    
    
    
	#Find Max contour area (Assume that hand is in the frame)
    max_area=100
    ci=0	
    for i in range(len(contours)):
        cnt=contours[i]
        area = cv2.contourArea(cnt)
        if(area>max_area):
            max_area=area
            ci=i  
    
    #Draw Contours
    cv2.drawContours(frame, cnt, -1, (122,122,0), 3)
    
    try:
        cnts = contours[ci]
    except IndexError:
        print("No contour found")
        continue
    #Largest area contour  
    cnts = contours[ci]

    #Find convex hull
    hull = cv2.convexHull(cnts)
    
    #Find moments of the largest contour
    moments = cv2.moments(cnts)
    
    #Find convex defects
    hull2 = cv2.convexHull(cnts,returnPoints = False)
    defects = cv2.convexityDefects(cnts,hull2)

#    cv2.drawContours(frame,[hull],-1,(255,255,255),2)
    
    #Get defect points and draw them in the original image
    FarDefect = []
    for i in range(defects.shape[0]):
        s,e,f,d = defects[i,0]
        start = tuple(cnts[s][0])
        end = tuple(cnts[e][0])
        far = tuple(cnts[f][0])
        FarDefect.append(far)
        cv2.line(frame,start,end,[0,255,0],1)
        cv2.circle(frame,far,10,[100,255,255],3)
    
    #Central mass of first order moments
    if moments['m00']!=0:
        cx = int(moments['m10']/moments['m00']) # cx = M10/M00
        cy = int(moments['m01']/moments['m00']) # cy = M01/M00
    centerMass=(cx,cy)    
    
    #Draw center mass
    cv2.circle(frame,centerMass,7,[100,0,255],2)
    font = cv2.FONT_HERSHEY_SIMPLEX
#     cv2.putText(frame,'Center',tuple(centerMass),font,2,(255,255,255),2)     
          
    #Print bounding rectangle
    x,y,w,h = cv2.boundingRect(cnts)
    img = cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)

    # Show final image
    cv2.imshow('Final result',frame)
    
    #Print execution time
#     print(time.time()-start_time)
    
    #close the output video by pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour found
No contour fou

NameError: name 'cap' is not defined